<a href="https://colab.research.google.com/github/taylorfogarty/launch/blob/master/neural_net_pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from collections import Counter
from google.colab import files
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
uploaded = files.upload()

In [0]:
data = pd.read_csv("pokemon.csv")
data['percentage_male'] = data['percentage_male'].fillna(50)
data = data.fillna(0)
y = data['generation']
X = data.drop(['abilities','type2','name','pokedex_number','generation','is_legendary'], axis=1)
X = pd.get_dummies(X)
X1 = data[['name','type1','generation','is_legendary','type2','hp','attack','defense']]

Supervised Learning

In [8]:
def baseline_model():
	model = Sequential()
	model.add(Dense(20, input_dim=50, activation='relu'))
	model.add(Dense(7, activation='sigmoid'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=614)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 18.50% (3.70%)


Unsupervised Learning

In [0]:
from sklearn.cluster import KMeans
k = 10
kmeans = KMeans(n_clusters=k).fit(X)
X1['group'] = pd.DataFrame(kmeans.labels_)

In [0]:
X4 = data[['name','type1','generation','is_legendary','type2','hp','sp_attack','sp_defense']]
clusters = AgglomerativeClustering(n_clusters=10).fit(X)
X4['cluster'] = clusters.labels_
X4 = X4.sort_values(by='cluster')

In [68]:
pca = PCA(n_components=20,svd_solver='full')
X2 = pca.fit_transform(X)
X3 = pd.DataFrame(pca.components_,columns=X.columns)
X3.shape

(20, 50)